In [203]:
from pathlib import Path
import os
import sys
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
from utils.crossvalidation import xval
from settings.paths import match_path, img_path, rf_path, bnn_path, flex_path
import pandas as pd
import matplotlib.pyplot as plt
from utils.preprocessing import create_bins, prep_data, rename_aper
from settings.columns import specz, calculate_colors, create_colors, list_feat
from utils.metrics import sigma, bias ,out_frac, rmse, mse
import warnings
import numpy as np
from utils.metrics import sigma, out_frac, bias, rmse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from cycler import cycler
from sklearn.model_selection import GridSearchCV
from utils.preprocessing import create_bins
from utils.crossvalidation import metric_per_bin
from utils.preprocessing import create_bins, rename_aper, mag_redshift_selection, prep_wise, missing_input

plt.rcParams["font.size"] = 22
blue = (0, 0.48, 0.70)
orange = (230/255,159/255, 0)
yellow = (0.94, 0.89, 0.26)
pink = (0.8, 0.47, 0.65)
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['axes.prop_cycle'] = cycler('color', CB_color_cycle)

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from utils.correct_extinction import correction
aper = "PStotal"

In [9]:
table_dr2 = pd.read_table(os.path.join(match_path,"STRIPE82_DR2_DR16Q1a_unWISE2a_GALEXDR672a.csv"), sep=",")
table_dr2 = rename_aper(table_dr2)

In [204]:
table_dr4 = pd.read_table(os.path.join(match_path,"STRIPE82_DR4_DR16Q1a_unWISE2a_GALEXDR672a.csv"), sep=",")
table_dr4 = rename_aper(table_dr4)

In [205]:
data = mag_redshift_selection(table_dr4, rmax=22, zmax=5)

In [206]:
data = prep_wise(data)


In [208]:
data = missing_input(data)


In [209]:
data, bins, itvs = create_bins(data = data, bin_size=0.5, return_data = True, var = specz)
train, test = train_test_split(data, random_state=47, stratify=data['Zclass'])
zclass_train = train["Zclass"]

train = correction(train)
test = correction(test)

train = calculate_colors(train, broad = True, narrow= True, wise = True, galex= True, aper=aper)
test =  calculate_colors(test, broad = True, narrow= True, wise = True, galex= True, aper=aper)

In [190]:
def fraction_of_non_observation(df, survey="WISE"):
    if survey=="WISE":
        print(len(df[df["objID_x"].isna()])/len(df))
        return 
    
    if survey=="GALEX":
        print(len(df[df["name"].isna()])/len(df))
        return 
    
def fraction_of_non_detection(df, survey="WISE"):
    if survey=="WISE":
        print("W1:", len(df[df["W1"].isna()])/len(df))
        print("W2:", len(df[df["W2"].isna()])/len(df))
        return 
    
    if survey=="GALEX":
        print("FUV:", len(df[df["FUVmag"].isna()].dropna(subset=["name"]))/len(df.dropna(subset=["name"])))
        print("NUV:", len(df[df["NUVmag"].isna()].dropna(subset=["name"]))/len(df.dropna(subset=["name"])))
        return

In [180]:
#number of objects with non-observation in GALEX
len(data[data["name"].isna()])

#number of objects with non-observation in WISE
len(data[data["objID_x"].isna()])

22940

In [197]:

feat_mag = list_feat(aper=aper, broad = True, narrow = True, galex = True, wise = True)
feat = create_colors(broad = True, narrow=True, wise=True, galex=True, aper=aper)
# data = table_corrected.copy(deep=True)

In [192]:
fraction_of_non_observation(data, survey="WISE")
fraction_of_non_observation(train, survey="WISE")
fraction_of_non_observation(test, survey="WISE")

0.060480830140870565
0.060129509713228495
0.06153474903474904


In [193]:
fraction_of_non_observation(data, survey="GALEX")
fraction_of_non_observation(train, survey="GALEX")
fraction_of_non_observation(test, survey="GALEX")

0.6919851588187385
0.6918312351687246
0.6924469111969112


In [194]:
fraction_of_non_detection(data, survey="WISE")
fraction_of_non_detection(train, survey="WISE")
fraction_of_non_detection(test, survey="WISE")

W1: 0.06922868088443787
W2: 0.14132303701245816
W1: 0.06901821984474922
W2: 0.14097252946144873
W1: 0.06986003861003862
W2: 0.14237451737451737


In [195]:
fraction_of_non_detection(data, survey="GALEX")
fraction_of_non_detection(train, survey="GALEX")
fraction_of_non_detection(test, survey="GALEX")

FUV: 0.6410733522671629
NUV: 0.0
FUV: 0.6378230227094753
NUV: 0.0
FUV: 0.6508434680266771
NUV: 0.0
